In [1]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianModel
import pgmpy.inference

In [2]:
def normalizeKindAge(age):
    if age == "n.a.":
        return "n.a.";
    elif int(age) > 18:
        return ">18"
    else:
        return "<18"

def normalizeIncome(income):
    income = int(income)
    if income < 20000:
        return "<20000"
    elif income < 40000:
        return "<40000"
    elif income < 60000:
        return "<60000"
    elif income < 80000:
        return "<80000"
    elif income < 100000:
        return "<100000"
    else:
        return ">100000"

In [3]:
data = pd.read_csv('versicherung_x.csv', delimiter=';')
data['aeltestesKind'] = data['aeltestesKind'].apply(normalizeKindAge)
data['juengstesKind'] = data['juengstesKind'].apply(normalizeKindAge)
data['Familieneinkommen'] = data['Familieneinkommen'].apply(normalizeIncome)

In [4]:
net = [
("Altersgruppe", "Bildungsstand"),
("Altersgruppe", "aeltestesKind"),
("Altersgruppe", "juengstesKind"),
("Altersgruppe", "Wohnverhaeltnis"),
("Altersgruppe", "Tarif"),

("Geschlecht", "Tarif"),

("Verheiratet", "Beruf"),
("Verheiratet", "Kinderzahl"),

("Kinderzahl", "Wohnverhaeltnis"),
("Kinderzahl", "Tarif"),

("aeltestesKind", "Wohnverhaeltnis"),

("juengstesKind", "Wohnverhaeltnis"),

("Bildungsstand", "Beruf"),
("Bildungsstand", "Aktienbesitz"),

("Beruf", "Familieneinkommen"),

("Familieneinkommen", "Tarif"),
("Familieneinkommen", "Wohnverhaeltnis"),

("Wohnverhaeltnis", "Tarif"),

("Ort", "Land"),
("Ort", "Wohnverhaeltnis"),

("Land", "Tarif"),

("Aktienbesitz", "Familieneinkommen")
]
model = BayesianModel(net)

In [5]:
separation = 191
train = data[:separation]
test = data[separation:]
model.fit(train)

In [6]:
result = model.predict(test.drop("Tarif", 1))

In [7]:
result["Expected"] = data[separation:]["Tarif"]
result

,Tarif,Expected
191,B,B
192,A,B
193,B,B
194,A,A
195,A,B
196,A,A
197,B,B
198,A,B
199,A,B
200,B,B


In [8]:
tests = len(result)
hits = np.where(result["Expected"] == result["Tarif"], [1 for x in range(tests)], [0 for x in range(tests)])
sum(hits) / tests

0.6